# Week 02 Assignment glucose level data


Missing data occurs commonly in many data applications. Especially wearable sensor devices encounter measurement errors resulting in missing data. Although pandas can handle missing data, for instance, to exclude missing data in the descriptive analysis, we might decide to reshape our data to improve quality. We might consider excluding certain rows, columns, or decide to impute our data if we take an argumentative approach. This week we will work with sensor data of a semester 01 project. The data is timeseries related but a lot of sensor errors occurred. First, we inspect the quality data. Then we will work with imputation, interpolation, and smoothing techniques to overcome this issue of the error readings. Finaly we plot the data. Visualization is a method to translate data into information. As a data scientist you should be able to choose an appropiate visualizations. 


Keywords: data loading, data inspection, data exploration, data cleaning, impute data, missing data, timeseries, pandas, visualization, interactive plots, overflow/underflow error

More to read
- https://fennaf.gitbook.io/bfvm22prog1/
- https://towardsdatascience.com/how-to-analyze-blood-glucose-data-with-python-data-science-packages-4f160f9564be


More about pandas and bokeh programming
- https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.interpolate.html
- https://rubberduckdebugging.com/
- https://docs.bokeh.org/en/latest/docs/gallery.html


Learning objectives

- Further practice numpy / pandas
- Read, inspect, clean, and impute JSON data. Handle missing data
- Apply different interpolation/smoothing techniques
- Recognize overflow/underflow error
- Perform visual and statistical analysis for time series data
- Maintain development environment 
- Apply coding standards and FAIR principles


Note: you can bring your own sensor data, but for the practical you need some missing data points. 

By the end of this week the student can:

- load a json dataset 
- typecast the Pandas DataFrame to appropiate data types
- inspect the dataset for quality and metadata information
- add a column with interpolated data in Pandas DataFrame
- perform visual analysis

please add your own topics you want to learn here: https://padlet.com/ffeenstra1/69vbvy0l8t32rq55


The assignment consists of 6 parts:

- [part 1: load the data](#0)
     - [Exercise 1.1](#ex-11)
- [part 2: prepare for inspection](#1)
     - [Exercise 2.1](#ex-21)
- [part 3: inspect the data](#2)
     - [Exercise 3.1](#ex-31)
- [part 4: interpolate the data](#3)
     - [Exercise 4.1](#ex-41)
- [part 5: visualize the data](#4)
     - [Exercise 5.1](#ex-51)
- [part 6: Challenge](#5)
     - [Exercise 6.1](#ex-61)

Part 1 and 5 are mandatory, part 6 is optional (bonus)
Mind you that you cannot copy code without referencing the code. If you copy code you need to be able to explain your code verbally and you will not get the full score. 


<a name='0'></a>
## Part 1: Load the data

Instructions: Load the json datafile `glucose.json` into a pandas dataframe. Check your dataframe with a `.head()` to compare with the expected outcome. 

Preferably we read the data not with a hard coded data path but using a config file. See https://fennaf.gitbook.io/bfvm22prog1/data-processing/configuration-files/yaml

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>json.load() method reads a file, pd.read_json converts it to a Pandas DataFrame</li>
    <li>when loading into a Pandas DataFrame use records orientation </li>
</ul>
</details>

<a name='ex-11'></a>
### Code your solution

In [1]:
# CODE YOUR SOLUTION HERE
import pandas as pd
import numpy as np
import yaml
import json

config = open("/homes/memartinus/Desktop/programming1/config.yml","r")
dictionary = yaml.safe_load(config)
path = open(dictionary["Path"])

loaded_file = json.load(path)
df = pd.read_json(loaded_file)
path.close()

df.head()



,ID,time,recordtype,glucose
0,2845.0,2019-04-25 00:08,1,109
1,2850.0,2019-04-25 00:50,1,
2,2877.0,2019-04-25 07:02,1,123
3,2881.0,2019-04-25 07:34,1,158
4,2886.0,2019-04-25 08:19,1,


#### Expected outcome: 

<a name='1'></a>
## Part 2: Prepare the data

Check the datatypes of your dataframe. The `glucose` field should be an integer, the `time` field should have a datetime format. If the datatypes are different you should typecast them to the right format.
Make sure that your dataset is sorted by the time column


<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>use astype() method or pandas.DataFrame.to_datetime() for instance</li>
    <li>make sure that the empty spaces are filled with NaN. Use errors='coerce'</li>
    <li>set_index(), sort_index() and reset_index() are helpful to sort on index</li>
</ul>
</details>

<a name='ex-21'></a>
### Code your solution

In [2]:
# CODE YOUR SOLUTION HERE
# first replace everyting with NAN if empty

df  = df.replace(r"^\s*$",np.nan,regex= True)
df["glucose"] = df["glucose"].astype("float")
df["time"] = pd.to_datetime(df["time"])
df = df.set_index(df["time"])



#### Expected outcome: 

<a name='2'></a>
## Part 3: Inspect the data

Now that we prepared the data we are going to inspect the data to get more familiar with the data. You can answer the questions below but feel free to explore the data further by plotting and or statistics, group by overviews and so on. 

To be answered at least:
- what is the quantity of the data (observations and features)
- what is the percentage missing data for glucose?
- is there a specific pattern for glucose value and recordtype?
- what is the time range?

Code the solutions to your answers. Create meaningful overviews or statistics

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>In the week 01 assignment some functions were explained to inspect missing values</li>
    <li>In the week 01 assignment some functions were explained to groupby value</li>
</ul>
</details>

<a name='ex-31'></a>
### Code your solution

In [3]:
#CODE YOUR SOLUTION HERE
obs = df.groupby(["ID"]).sum().count()
features = len(df.columns)
mv = df["glucose"].isnull().sum()
total = mv + df["glucose"].count()
print("number of features:", features)
print("number of observations:",obs)
print("percentage of missing values:",mv/total)


number of features: 4
number of observations: recordtype    56
glucose       56
dtype: int64
percentage of missing values: 0.6176470588235294


#### Expected outcome percentage missing data
0.6176470588235294

<a name='3'></a>
## Part 4: Interpolate the data

A lot of data is missing. Use interpolation to fill the missing values. Create a new column with the interpolated data. Take an argumentative approach. Select an interpolation method that suits the nature of the data and explain your choice. Mind you that the expected outcome of the interpolation values can differ from the example below

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>use Pandas.DataFrame.interpolate() method</li>
</ul>
</details>

<a name='ex-41'></a>
### Code your solution

In [4]:

# slinear was used because linear doesnt take in to consideration the time/date but uses a regular linspace to splice
# the numbers. also single linear uses de first order to plot the graph.
df["interpolated"] = df['glucose'].interpolate(method = "slinear")
df = df.sort_index()
df.head(8)

,ID,time,recordtype,glucose,interpolated
time,,,,,
2019-04-25 00:08:00,2.845000e+03,2019-04-25 00:08:00,1,109.0,109.000000
2019-04-25 00:14:00,1.614305e+19,2019-04-25 00:14:00,0,NaN,109.202899
2019-04-25 00:29:00,1.614305e+19,2019-04-25 00:29:00,0,NaN,109.710145
2019-04-25 00:44:00,1.614305e+19,2019-04-25 00:44:00,0,NaN,110.217391
2019-04-25 00:50:00,2.850000e+03,2019-04-25 00:50:00,1,NaN,110.420290
2019-04-25 00:59:00,1.614305e+19,2019-04-25 00:59:00,0,NaN,110.724638
2019-04-25 01:14:00,1.614305e+19,2019-04-25 01:14:00,0,NaN,111.231884
2019-04-25 01:29:00,1.614305e+19,2019-04-25 01:29:00,0,NaN,111.739130


#### Example outcome

<a name='4'></a>
## Part 5: Plot the data

Create a plot with the original data and the interpolated data. Consider what the best representation is for visualisation of actual values and modelled/imputed values. Provide argumentation for the plot of choice. 

Mind you, there are several types of plots. The most used are: 
- Compare values->	Bar, boxplot, violin plot, scatter, line. 
- Show composition-> Stacked bar, pie, mosaic, area, burtin.
- Show distribution-> 	Histogram, density, qqplot, bubble. 
- Analyzing trend-> Line, bar, dual axis 
- Relations-> Scatter, heatmap


### Code your solution

In [5]:
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.layouts import *
from bokeh.io import output_notebook
from bokeh.plotting import ColumnDataSource
output_notebook()

Loading BokehJS ...

In [6]:
#CODE YOUR SOLUTION HERE

graph = figure(title="awesome streep van glucose")
graph.yaxis.axis_label = "glucose units"
graph.xaxis.axis_label = "time"
legend_label = "interpolated values"
legend_label1 = "standard values"
graph.circle(df["time"], df["glucose"] , color = "blue", size = 10,legend_label = legend_label1)
graph.line(df["time"], df["interpolated"] , color = "red", legend_label = legend_label )

show(graph)

<a name='6'></a>
## Part 6: Challenge

It might even be interesting to introduce a widget in which you can select different methods to interpolate.
1. Can you improve the interpolation by choosing an other method?
2. Can you add an rolling mean line? 
2. Can you improve the plot by making it interactive?

<a name='ex-61'></a>
### Code your solution

In [7]:
#CODE YOUR SOLUTION HERE
import panel as pn



